In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install neptune-client

     |████████████████████████████████| 245kB 26.1MB/s 
     |████████████████████████████████| 829kB 31.7MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
     |████████████████████████████████| 174kB 40.5MB/s 
     |████████████████████████████████| 133kB 49.7MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.9.13-py2.py3-none-any.whl size=417559 sha256=24cea6b76fe8ecdf641ddc9cc789a68820d774baaf1a9dce64b55d30aa027a99
  Stored in directory: /root/.cache/pip/wheels/8f/d2/41/e720f9f79300f6bda2a8bf8cb313e108d51d0cb9f130630901
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=0170fe0d352b3b1e65febd83477595320f6854fe4fee7bca0bef606219e37011
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp37-none-a

In [19]:
#%%

import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.metrics import plot_confusion_matrix

In [4]:
import neptune

API_TOKEN='' #@param {type:"string"}

PROJEKT_NAME = 'DataWorkshop-Foundation/Predicting-cancellation-of-visit' 

neptune.init(
   api_token=API_TOKEN,
   project_qualified_name=PROJEKT_NAME
)

Project(DataWorkshop-Foundation/Predicting-cancellation-of-visit)

In [58]:
train = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Cancellation/data/EDA2/train.csv' )
test = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Cancellation/data/EDA2/test.csv')
validation = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/Cancellation/data/EDA2/validation.csv')

In [59]:
train

,Unnamed: 0,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,Neighbourhood_7,Neighbourhood_8,Neighbourhood_9,Neighbourhood_10,Neighbourhood_11,Neighbourhood_12,Neighbourhood_13,Neighbourhood_14,Neighbourhood_15,Neighbourhood_16,Neighbourhood_17,Neighbourhood_18,Neighbourhood_19,Neighbourhood_20,Neighbourhood_21,Neighbourhood_22,Neighbourhood_23,Neighbourhood_24,Neighbourhood_25,Neighbourhood_26,Neighbourhood_27,Neighbourhood_28,Neighbourhood_29,Neighbourhood_30,Neighbourhood_31,Neighbourhood_32,Neighbourhood_33,...,Neighbourhood_53,Neighbourhood_54,Neighbourhood_55,Neighbourhood_56,Neighbourhood_57,Neighbourhood_58,Neighbourhood_59,Neighbourhood_60,Neighbourhood_61,Neighbourhood_62,Neighbourhood_63,Neighbourhood_64,Neighbourhood_65,Neighbourhood_66,Neighbourhood_67,Neighbourhood_68,Neighbourhood_69,Neighbourhood_70,Neighbourhood_71,Neighbourhood_72,Neighbourhood_73,Neighbourhood_74,Neighbourhood_75,Neighbourhood_76,Neighbourhood_77,Neighbourhood_78,Neighbourhood_79,Neighbourhood_80,Neighbourhood_81,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,Waiting_Time_days,ScheduledHour,ScheduledDay_in_year,AppointmentDay_in_year
0,36775,79663627249177,5683597,1,2016-05-11 07:40:04,2016-05-16 00:00:00,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,7,132,137
1,36776,718169788381286,5683814,0,2016-05-11 07:54:03,2016-05-18 00:00:00,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,7,7,132,139
2,36777,3139218535988,5711485,1,2016-05-18 07:41:09,2016-05-25 00:00:00,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,7,7,139,146
3,36779,117342186564746,5415492,1,2016-03-01 13:23:59,2016-05-04 00:00:00,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,64,13,61,125
4,36780,54139872518,5599591,1,2016-04-19 09:09:23,2016-05-04 00:00:00,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,15,9,110,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55286,110521,36355337746436,5651072,1,2016-05-03 08:23:40,2016-06-07 00:00:00,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,35,8,124,159
55287,110522,2572134369293,5651768,1,2016-05-03 09:15:35,2016-06-07 00:00:00,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,35,9,124,159
55288,110524,15576631729893,5630692,1,2016-04-27 16:03:52,2016-06-07 00:00:00,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,41,16,118,159
55289,110525,92134931435557,5630323,1,2016-04-27 15:09:23,2016-06-07 00:00:00,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,41,15,118,159


In [60]:
test

,Unnamed: 0,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,Neighbourhood_7,Neighbourhood_8,Neighbourhood_9,Neighbourhood_10,Neighbourhood_11,Neighbourhood_12,Neighbourhood_13,Neighbourhood_14,Neighbourhood_15,Neighbourhood_16,Neighbourhood_17,Neighbourhood_18,Neighbourhood_19,Neighbourhood_20,Neighbourhood_21,Neighbourhood_22,Neighbourhood_23,Neighbourhood_24,Neighbourhood_25,Neighbourhood_26,Neighbourhood_27,Neighbourhood_28,Neighbourhood_29,Neighbourhood_30,Neighbourhood_31,Neighbourhood_32,Neighbourhood_33,...,Neighbourhood_53,Neighbourhood_54,Neighbourhood_55,Neighbourhood_56,Neighbourhood_57,Neighbourhood_58,Neighbourhood_59,Neighbourhood_60,Neighbourhood_61,Neighbourhood_62,Neighbourhood_63,Neighbourhood_64,Neighbourhood_65,Neighbourhood_66,Neighbourhood_67,Neighbourhood_68,Neighbourhood_69,Neighbourhood_70,Neighbourhood_71,Neighbourhood_72,Neighbourhood_73,Neighbourhood_74,Neighbourhood_75,Neighbourhood_76,Neighbourhood_77,Neighbourhood_78,Neighbourhood_79,Neighbourhood_80,Neighbourhood_81,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,Waiting_Time_days,ScheduledHour,ScheduledDay_in_year,AppointmentDay_in_year
0,7278,293397656722629,5645167,0,2016-05-02 09:33:26,2016-05-12 00:00:00,36,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,10,9,123,133
1,7280,2691555495224,5596589,0,2016-04-18 14:54:52,2016-05-12 00:00:00,63,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,24,14,109,133
2,7283,9981638424867,5554863,1,2016-04-07 09:08:54,2016-05-05 00:00:00,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,28,9,98,126
3,7286,667141493898633,5576959,1,2016-04-13 09:14:12,2016-05-12 00:00:00,65,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,29,9,104,133
4,7287,216693861728386,5493935,0,2016-03-21 09:18:16,2016-05-19 00:00:00,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,9,81,140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38116,110498,94336536145654,5787655,1,2016-06-08 10:21:14,2016-06-08 00:00:00,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,160,160
38117,110503,23621816822757,5759473,1,2016-06-01 13:00:36,2016-06-01 00:00:00,64,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,153,153
38118,110507,476946211846992,5786918,1,2016-06-08 09:04:18,2016-06-08 00:00:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,160,160
38119,110508,94336536145654,5757656,1,2016-06-01 09:41:00,2016-06-01 00:00:00,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,153,153


In [61]:
validation

,Unnamed: 0,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,Neighbourhood_7,Neighbourhood_8,Neighbourhood_9,Neighbourhood_10,Neighbourhood_11,Neighbourhood_12,Neighbourhood_13,Neighbourhood_14,Neighbourhood_15,Neighbourhood_16,Neighbourhood_17,Neighbourhood_18,Neighbourhood_19,Neighbourhood_20,Neighbourhood_21,Neighbourhood_22,Neighbourhood_23,Neighbourhood_24,Neighbourhood_25,Neighbourhood_26,Neighbourhood_27,Neighbourhood_28,Neighbourhood_29,Neighbourhood_30,Neighbourhood_31,Neighbourhood_32,Neighbourhood_33,...,Neighbourhood_53,Neighbourhood_54,Neighbourhood_55,Neighbourhood_56,Neighbourhood_57,Neighbourhood_58,Neighbourhood_59,Neighbourhood_60,Neighbourhood_61,Neighbourhood_62,Neighbourhood_63,Neighbourhood_64,Neighbourhood_65,Neighbourhood_66,Neighbourhood_67,Neighbourhood_68,Neighbourhood_69,Neighbourhood_70,Neighbourhood_71,Neighbourhood_72,Neighbourhood_73,Neighbourhood_74,Neighbourhood_75,Neighbourhood_76,Neighbourhood_77,Neighbourhood_78,Neighbourhood_79,Neighbourhood_80,Neighbourhood_81,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,Waiting_Time_days,ScheduledHour,ScheduledDay_in_year,AppointmentDay_in_year
0,0,29872499824296,5642903,1,2016-04-29 18:38:08,2016-04-29 00:00:00,62,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,18,120,120
1,1,558997776694438,5642503,0,2016-04-29 16:08:27,2016-04-29 00:00:00,56,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,120,120
2,2,4262962299951,5642549,1,2016-04-29 16:19:04,2016-04-29 00:00:00,62,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,120,120
3,3,867951213174,5642828,1,2016-04-29 17:29:31,2016-04-29 00:00:00,8,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,120,120
4,4,8841186448183,5642494,1,2016-04-29 16:07:23,2016-04-29 00:00:00,56,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,16,120,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17108,110499,821969177626116,5757697,1,2016-06-01 09:42:56,2016-06-01 00:00:00,66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,9,153,153
17109,110500,443438443334614,5787233,1,2016-06-08 09:35:13,2016-06-08 00:00:00,59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,160,160
17110,110505,56673438855979,5758455,1,2016-06-01 10:45:50,2016-06-01 00:00:00,55,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,153,153
17111,110519,65423877893936,5630447,1,2016-04-27 15:23:14,2016-06-07 00:00:00,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,41,15,118,159


In [62]:
#%%
train.drop(['Unnamed: 0', 'PatientId', 'AppointmentID','ScheduledDay',
       'AppointmentDay'],  axis = 1, inplace =True)
test.drop(['Unnamed: 0','PatientId', 'AppointmentID','ScheduledDay',
       'AppointmentDay'], axis = 1, inplace =True)
validation.drop(['Unnamed: 0','PatientId', 'AppointmentID','ScheduledDay',
       'AppointmentDay'], axis = 1, inplace =True)

In [63]:
X_train = train.loc[:, train.columns !='No-show']
y_train = train.loc[:, train.columns =='No-show']

#%%
X_test = test.loc[:, test.columns !='No-show']
y_test = test.loc[:, test.columns =='No-show']

#%%
X_val = validation.loc[:, validation.columns !='No-show']
y_val =  validation.loc[:, validation.columns =='No-show']

In [64]:
X_train

,Gender,Age,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,Neighbourhood_7,Neighbourhood_8,Neighbourhood_9,Neighbourhood_10,Neighbourhood_11,Neighbourhood_12,Neighbourhood_13,Neighbourhood_14,Neighbourhood_15,Neighbourhood_16,Neighbourhood_17,Neighbourhood_18,Neighbourhood_19,Neighbourhood_20,Neighbourhood_21,Neighbourhood_22,Neighbourhood_23,Neighbourhood_24,Neighbourhood_25,Neighbourhood_26,Neighbourhood_27,Neighbourhood_28,Neighbourhood_29,Neighbourhood_30,Neighbourhood_31,Neighbourhood_32,Neighbourhood_33,Neighbourhood_34,Neighbourhood_35,Neighbourhood_36,Neighbourhood_37,Neighbourhood_38,...,Neighbourhood_52,Neighbourhood_53,Neighbourhood_54,Neighbourhood_55,Neighbourhood_56,Neighbourhood_57,Neighbourhood_58,Neighbourhood_59,Neighbourhood_60,Neighbourhood_61,Neighbourhood_62,Neighbourhood_63,Neighbourhood_64,Neighbourhood_65,Neighbourhood_66,Neighbourhood_67,Neighbourhood_68,Neighbourhood_69,Neighbourhood_70,Neighbourhood_71,Neighbourhood_72,Neighbourhood_73,Neighbourhood_74,Neighbourhood_75,Neighbourhood_76,Neighbourhood_77,Neighbourhood_78,Neighbourhood_79,Neighbourhood_80,Neighbourhood_81,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,Waiting_Time_days,ScheduledHour,ScheduledDay_in_year,AppointmentDay_in_year
0,1,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,7,132,137
1,0,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,7,7,132,139
2,1,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,7,7,139,146
3,1,58,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,64,13,61,125
4,1,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,15,9,110,125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55286,1,53,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,35,8,124,159
55287,1,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,35,9,124,159
55288,1,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,41,16,118,159
55289,1,38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,41,15,118,159


In [65]:
y_train

,No-show
0,0
1,0
2,0
3,0
4,0
...,...
55286,0
55287,0
55288,0
55289,0


In [70]:
def save_metrics(dataset, name, tags):
  neptune.create_experiment(
      name=name
  )

  # model = train_model(model, X_train, y_train)
  # y_pred = predict_model(model, X_test)
  # val_pred = predict_model(model, X_val)

  accuracy = accuracy_score(dataset['iforest'], dataset['No-show'])
  recall = recall_score(dataset['iforest'], dataset['No-show'])
  precision = precision_score(dataset['iforest'], dataset['No-show'])
  f1 = f1_score(dataset['iforest'], dataset['No-show'])
  try:
    roc = roc_auc_score(dataset['iforest'], dataset['No-show'])
  except ValueError:
    pass
  print("Test")
  print(dataset['No-show'],'\n')
  print('accuracy: ', accuracy)
  print('recall: ',recall)
  print('precision: ', precision)
  print('f1: ', f1)
  try:
    print('ROC AUC: ', roc,'\n')
  except UnboundLocalError:
    pass
  print(classification_report(dataset['iforest'], dataset['No-show']))

  neptune.log_metric('accuracy', accuracy)
  neptune.log_metric('recall', recall)
  neptune.log_metric('precision', precision)
  neptune.log_metric('f1', f1)
  try: 
    neptune.log_metric('roc_auc', roc)
  except UnboundLocalError:
    pass
  neptune.append_tags(tags)
  neptune.append_tags(name)
  neptune.append_tags(["Magdalena Cebula"])

  neptune.stop()

In [72]:
X_train2 = X_train.copy()

In [71]:
model = IsolationForest(random_state=0)
#%%
## Fitting the model 
model.fit(X_train)
#%%
y_train['iforest'] =pd.Series(model.predict(X_train))
#%%
y_train['iforest'] = y_train['iforest'].map({1:0, -1:1})
print(y_train['iforest'].value_counts())

save_metrics(y_train, 'IsolationForest', ['ISF', 'EDA2', 'contamination=auto', 'train'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0    55291
Name: iforest, dtype: int64
https://ui.neptune.ai/DataWorkshop-Foundation/Predicting-cancellation-of-visit/e/PCV-683
Test
0        0
1        0
2        0
3        0
4        0
        ..
55286    0
55287    0
55288    0
55289    0
55290    0
Name: No-show, Length: 55291, dtype: int64 

accuracy:  0.8017760575862256
recall:  0.0
precision:  0.0
f1:  0.0
              precision    recall  f1-score   support

           0       1.00      0.80      0.89     55291
           1       0.00      0.00      0.00         0

    accuracy                           0.80     55291
   macro avg       0.50      0.40      0.44     55291
weighted avg       1.00      0.80      0.89     55291



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
#%% 
#próba 2
y_train2 = train.loc[:, train.columns =='No-show']
model = IsolationForest(random_state=0, contamination = 0.25)
model.fit(X_train2)

y_train2['iforest'] =pd.Series(model.predict(X_train))

y_train2['iforest'] = y_train2['iforest'].map({1:0, -1:1})
print(y_train2['iforest'].value_counts())

save_metrics(y_train2, 'IsolationForest', ['ISF', 'EDA2', 'contamination=0.25', 'train'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


0    41468
1    13823
Name: iforest, dtype: int64
https://ui.neptune.ai/DataWorkshop-Foundation/Predicting-cancellation-of-visit/e/PCV-684
Test
0        0
1        0
2        0
3        0
4        0
        ..
55286    0
55287    0
55288    0
55289    0
55290    0
Name: No-show, Length: 55291, dtype: int64 

accuracy:  0.6627118337523286
recall:  0.22187658250741518
precision:  0.27983576642335767
f1:  0.2475083726748174
ROC AUC:  0.5157685217929185 

              precision    recall  f1-score   support

           0       0.76      0.81      0.78     41468
           1       0.28      0.22      0.25     13823

    accuracy                           0.66     55291
   macro avg       0.52      0.52      0.52     55291
weighted avg       0.64      0.66      0.65     55291



In [74]:
y_test2 = test.loc[:, test.columns =='No-show']
y_test2['iforest'] =pd.Series(model.predict(X_test))

y_test2['iforest'] = y_test2['iforest'].map({1:0, -1:1})
print(y_test2['iforest'].value_counts())

save_metrics(y_test2, 'IsolationForest', ['ISF', 'EDA2', 'contamination=0.25', 'test', 'learnTrainCheckTest'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0    30040
1     8081
Name: iforest, dtype: int64
https://ui.neptune.ai/DataWorkshop-Foundation/Predicting-cancellation-of-visit/e/PCV-685
Test
0        1
1        0
2        0
3        1
4        0
        ..
38116    0
38117    0
38118    0
38119    0
38120    0
Name: No-show, Length: 38121, dtype: int64 

accuracy:  0.6746412738385666
recall:  0.22794208637544858
precision:  0.23007744191856108
f1:  0.22900478647355008
ROC AUC:  0.5113745052383235 

              precision    recall  f1-score   support

           0       0.79      0.79      0.79     30040
           1       0.23      0.23      0.23      8081

    accuracy                           0.67     38121
   macro avg       0.51      0.51      0.51     38121
weighted avg       0.67      0.67      0.67     38121



In [75]:
y_val2 = validation.loc[:, validation.columns =='No-show']
y_val2['iforest'] =pd.Series(model.predict(X_val))

y_val2['iforest'] = y_val2['iforest'].map({1:0, -1:1})
print(y_val2['iforest'].value_counts())

save_metrics(y_val2, 'IsolationForest', ['ISF', 'EDA2', 'contamination=0.25', 'test', 'learnTrainCheckVal'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


0    12704
1     4409
Name: iforest, dtype: int64
https://ui.neptune.ai/DataWorkshop-Foundation/Predicting-cancellation-of-visit/e/PCV-686
Test
0        0
1        0
2        0
3        0
4        0
        ..
17108    0
17109    0
17110    0
17111    0
17112    0
Name: No-show, Length: 17113, dtype: int64 

accuracy:  0.6545900777186934
recall:  0.20979814016783851
precision:  0.2759546539379475
f1:  0.2383713438989821
ROC AUC:  0.5093779743660352 

              precision    recall  f1-score   support

           0       0.75      0.81      0.78     12704
           1       0.28      0.21      0.24      4409

    accuracy                           0.65     17113
   macro avg       0.51      0.51      0.51     17113
weighted avg       0.63      0.65      0.64     17113



In [82]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials
from hyperopt.pyll.base import scope
from sklearn.model_selection import cross_val_score

In [89]:
X_train3 = train.loc[:, train.columns !='No-show']
y_train3 = train.loc[:, train.columns =='No-show']

In [95]:
space = {
    "n_estimators": hp.choice("n_estimators", [100, 200, 300, 400,500,600]),
    "contamination": hp.quniform("contamination", 0.01, 0.3,0.05),
}

In [96]:
def hyperparameter_tuning(params):
    model = IsolationForest(**params,n_jobs=-1, random_state=0, warm_start=True, bootstrap=True)
    model.fit(X_train3)

    y_train3['iforest'] =pd.Series(model.predict(X_train))

    y_train3['iforest'] = y_train3['iforest'].map({1:0, -1:1})
    f1 = f1_score(y_train3['iforest'], y_train3['No-show'])

    return {"loss": -f1, "status": STATUS_OK}

In [97]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

  2%|▏         | 1/50 [00:18<14:53, 18.24s/it, best loss: -0.1896748727537135]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys



  4%|▍         | 2/50 [01:12<23:12, 29.02s/it, best loss: -0.1896748727537135]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys



  6%|▌         | 3/50 [01:30<20:10, 25.75s/it, best loss: -0.1896748727537135]

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys



 98%|█████████▊| 49/50 [24:23<00:30, 30.77s/it, best loss: -0.26826877699931023]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys



100%|██████████| 50/50 [24:58<00:00, 29.98s/it, best loss: -0.26826877699931023]
Best: {'contamination': 0.30000000000000004, 'n_estimators': 3}


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys



In [98]:
print("Best: {}".format(best))

Best: {'contamination': 0.30000000000000004, 'n_estimators': 3}


In [101]:
X_train4 = train.loc[:, train.columns !='No-show']
y_train4 = train.loc[:, train.columns =='No-show']

model = IsolationForest(random_state=0, n_jobs=-1, warm_start=True, bootstrap=True, contamination=0.3, n_estimators=400 )
model.fit(X_train4)
#%%
y_train4['iforest'] =pd.Series(model.predict(X_train4))
#%%
y_train4['iforest'] = y_train4['iforest'].map({1:0, -1:1})
print(y_train4['iforest'].value_counts())

save_metrics(y_train4, 'IsolationForest', ['ISF', 'EDA2', 'hyperopt', 'train'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


0    38704
1    16587
Name: iforest, dtype: int64
https://ui.neptune.ai/DataWorkshop-Foundation/Predicting-cancellation-of-visit/e/PCV-687
Test
0        0
1        0
2        0
3        0
4        0
        ..
55286    0
55287    0
55288    0
55289    0
55290    0
Name: No-show, Length: 55291, dtype: int64 

accuracy:  0.6354379555443019
recall:  0.22276481581961777
precision:  0.33713503649635035
f1:  0.26826877699931023
ROC AUC:  0.5175290594186969 

              precision    recall  f1-score   support

           0       0.71      0.81      0.76     38704
           1       0.34      0.22      0.27     16587

    accuracy                           0.64     55291
   macro avg       0.52      0.52      0.51     55291
weighted avg       0.60      0.64      0.61     55291

